<a href="https://colab.research.google.com/github/joomm/CNN/blob/main/PL%20CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [155]:
import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
import torch.optim as optim

# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 드라이브에 있는 압축 데이터 파일 풀기

%cd /content/drive/MyDrive

!unzip -qq "/content/drive/MyDrive/kdata/archive.zip"

/content/drive/MyDrive
replace Plants_2/images to predict/0001_0170.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

import glob

images = glob.glob('/content/drive/MyDrive/Plants_2/train/*.jpg')

filepaths = list(images)

len(filepaths)

|인덱스에 접근할 때 데이터 불러오기|

• 생성자에서 인수로 넘어온 data_dir의 파일들을 데이터로 사용하기 위해 리스트로 저장하고, 이미지를 텐서로 변환하는 transform 사용.

• __getitem__로 인덱스가 넘어오면, 경로에 해당하는 이미지를 불러와 텐서로 변환해 x_data로 반환.
• 타겟 데이터(y_data)는 파일 이름으로부터 추출
  - 식물 이름은 P0부터 P11까지 지정
  - 전체 데이터 세트를 0000부터 0022까지의 22개 주제 범주로 나눔
   (0000부터 0011까지는 건강한 클래스, 0012부터 0022까지는 질병 클래스)

• 이를 if else문으로 판단해 반환.



In [156]:
# 인덱스에 접근할 때 파일 경로로부터 데이터를 불러오는 데이터셋 클래스
# 건강한 잎인지 병든 잎인지 구별하기 위한 데이터셋

class PlantDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.class_to_idx = {'Alstonia Scholaris diseased (P2a)': 0,
                             'Alstonia Scholaris healthy (P2b)': 1,
                             'Arjun diseased (P1a)': 2,
                             'Arjun healthy (P1b)': 3,
                             'Bael diseased (P4b)': 4,
                             'Basil healthy (P8)': 5,
                             'Chinar diseased (P11b)': 6,
                             'Chinar healthy (P11a)': 7,
                             'Gauva diseased (P3b)': 8,
                             'Gauva healthy (P3a)': 9,
                             'Jamun diseased (P5b)': 10,
                             'Jamun healthy (P5a)': 11,
                             'Jatropha diseased (P6b)': 12,
                             'Jatropha healthy (P6a)': 13,
                             'Lemon diseased (P10b)': 14,
                             'Lemon healthy (P10a)': 15,
                             'Mango diseased (P0b)': 16,
                             'Mango healthy (P0a)': 17,
                             'Pomegranate diseased (P9b)': 18,
                             'Pomegranate healthy (P9a)': 19,
                             'Pongamia Pinnata diseased (P7b)': 20,
                             'Pongamia Pinnata healthy (P7a)': 21}
        self.classes = list(self.class_to_idx.keys())
        self.transform = transform

        self.samples = []
        for plant_folder in os.listdir(data_dir):
            if os.path.isdir(os.path.join(data_dir, plant_folder)):
                class_name = plant_folder
                for file in os.listdir(os.path.join(data_dir, plant_folder)):
                    path = os.path.join(data_dir, plant_folder, file)
                    target = self.class_to_idx[class_name]
                    self.samples.append((path, target))

    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = Image.open(path)
        if self.transform:
            sample = self.transform(sample)
        return sample, target

    def __len__(self):
        return len(self.samples)

학습에 사용할 파라미터 설정.

In [157]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [158]:
train_path = '/content/drive/MyDrive/Plants_2/train'  # 데이터가 있는 경로
test_path = '/content/drive/My Drive/Plants_2/test'

# 데이터셋에 적용할 변환 정의
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # 이미지 사이즈 설정
    transforms.ToTensor(),  # 텐서로 변환
])

데이터셋 불러오기

In [159]:
# ImageFolder를 사용하여 train 데이터 불러오기
train_dataset = PlantDataset(data_dir=train_path, transform=transform)

# ImageFolder를 사용하여 test 데이터 불러오기
test_dataset = PlantDataset(data_dir=test_path, transform=transform)

데이터로더를 사용하여 배치 크기를 지정

In [160]:
# 데이터로더 생성
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

클래스로 모델을 설계

In [161]:
import torch
import torch.nn as nn
# 배치 크기 × 채널 × 높이(height) × 너비(widht)의 크기의 텐서를 선언
inputs = torch.Tensor(100, 3, 64, 64) # 텐서의 크기 print('텐서의 크기 : {}'.format(inputs.shape))

conv1 = nn.Conv2d(3, 16, 5, padding=1)
print(conv1)

conv2 = nn.Conv2d(16, 32, kernel_size=5, padding=1)
print(conv2)

pool = nn.MaxPool2d(2)
print(pool)

# 각 레이어를 통과한 후 텐서의 크기
out = conv1(inputs)
print(out.shape)

out = pool(out)
print(out.shape)

out = conv2(out)
print(out.shape)

out = pool(out)
print(out.shape)


out.size(0) # 배치 크기 100
out.size(1) # 출력 채널의 크기 32
out.size(2) # 높이 14
out.size(3) # 너비 14

# 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼쳐라
out = out.view(out.size(0), -1)
print(out.shape)

fc = nn.Linear(6272, 100) # input_dim = 6,272, output_dim = 100
out = fc(out)
print(out.shape)

Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
torch.Size([100, 16, 62, 62])
torch.Size([100, 16, 31, 31])
torch.Size([100, 32, 29, 29])
torch.Size([100, 32, 14, 14])
torch.Size([100, 6272])
torch.Size([100, 100])


In [162]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5

        # L1 ImgIn shape=(?, 64, 64, 3)
        #    Conv     -> (?, 64, 64, 16)
        #    Pool     -> (?, 32, 32, 16)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)) # 이미지 크기 절반

        # L2 ImgIn shape=(?, 32, 32, 16)
        #    Conv     -> (?, 32, 32, 32)
        #    Pool     -> (?, 16, 16, 32)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # L3 ImgIn shape=(?, 16, 16, 32)
        #    Conv     -> (?, 16, 16, 64)
        #    Pool     -> (?, 8, 8, 64)
        self.layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))

        self.fc1 = torch.nn.Linear( 3136, 100, bias=True)
        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = torch.nn.Sequential(
            self.fc1,
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))

        # L5 Final FC 100 inputs -> 14 outputs
        self.fc2 = torch.nn.Linear(100, 14, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.layer4(out)
        out = self.fc2(out)
        return out

모델 정의

In [163]:
# CNN 모델 정의
model = CNN().to(device)

#비용 함수와 옵티마이저를 정의
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#총 배치의 수 출력
total_batch = len(train_dataloader)
print('총 배치의 수 : {}'.format(total_batch))   # 총 배치 수 42

총 배치의 수 : 42


총 배치의 수는 42입니다. 그런데 배치 크기를 100으로 했으므로 결국 훈련 데이터는 총 4,200개란 의미입니다. 이제 모델을 훈련시켜보겠습니다.

In [169]:
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_dataloader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

IndexError: ignored

In [166]:
# 손실 함수 및 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [167]:
# 데이터셋에 적용할 변환 정의
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),  # 텐서로 변환
])

# PlantDataset를 사용하여 train 데이터 불러오기
train_dataset = PlantDataset(data_dir=train_path, transform=transform)

# 클래스 확인
classes = train_dataset.classes

# 클래스 이름과 인덱스 매핑
class_to_idx = train_dataset.class_to_idx

# 클래스 이름 및 해당 인덱스 출력
for k, v in class_to_idx.items():
    print(f"Class: {k}, Index: {v}")

Class: Alstonia Scholaris diseased (P2a), Index: 0
Class: Alstonia Scholaris healthy (P2b), Index: 1
Class: Arjun diseased (P1a), Index: 2
Class: Arjun healthy (P1b), Index: 3
Class: Bael diseased (P4b), Index: 4
Class: Basil healthy (P8), Index: 5
Class: Chinar diseased (P11b), Index: 6
Class: Chinar healthy (P11a), Index: 7
Class: Gauva diseased (P3b), Index: 8
Class: Gauva healthy (P3a), Index: 9
Class: Jamun diseased (P5b), Index: 10
Class: Jamun healthy (P5a), Index: 11
Class: Jatropha diseased (P6b), Index: 12
Class: Jatropha healthy (P6a), Index: 13
Class: Lemon diseased (P10b), Index: 14
Class: Lemon healthy (P10a), Index: 15
Class: Mango diseased (P0b), Index: 16
Class: Mango healthy (P0a), Index: 17
Class: Pomegranate diseased (P9b), Index: 18
Class: Pomegranate healthy (P9a), Index: 19
Class: Pongamia Pinnata diseased (P7b), Index: 20
Class: Pongamia Pinnata healthy (P7a), Index: 21


테스트

In [171]:
# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = []
    Y_test = []
    for data, labels in test_dataloader:
        data = data.to(device)
        label = labels.to(device)
        X_test.append(data)
        Y_test.append(label)

    X_test = torch.cat(X_test, dim=0)
    Y_test = torch.cat(Y_test, dim=0)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.05999999865889549


층을 깊게 쌓는 것이 정확도를 올려주진 않음. 효율적으로 쌓는 것도 중요.